# GoProtect
@a_yordanova

В этом проекте мы разрабатываем решение для поиска совпадений названия спортивной школы, введённого пользователем, с эталонными названиями.

Декомпозиция задачи:
1. Загрузка и обзор данных.
2. Исследовательский анализ данных.
3. Подготовка данных.
4. Создание функций для применения моделей
    - Семантический поиск соответствий в списке эталонных названий для каждого варианта пользовательского ввода:
        - только по названию;
        - по названию и региону;
        - по полному названию без аббревиатур;
        - по полному названию с расшифрованными аббревиатурами.
    - Выбор лучшего совпадения по результатам всех проверок.
5. Оценка качества моделей.
6. Результаты исследования.

## 1. Загрузка и обзор данных

In [1]:
# Импортируем библиотеки
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
import heapq
from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer, util
from IPython.display import display



In [2]:
# Загружаем датасет с эталонными названиями
reference = pd.read_csv('D:/jupyter_notebooks/GoProtect/dataset/reference.csv')
display(reference.sample(10, random_state=42))
reference.info()

,school_id,name,region
182,183,СШ № 1,Нижегородская область
154,155,СК РОФФКК,Ростовская область
111,112,НЛФК,Свердловская область
203,204,СШОР № 1,Пермский край
60,61,Звезда,Санкт-Петербург
9,10,Академия синхронного катания на коньках,Москва
119,120,Олимп,Нижегородская область
157,158,Снеговик,Краснодарский край
167,168,Старт,Пермский край
33,34,Волкова,Москва


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


In [3]:
# Загружаем датасет с сырыми данными
raw_data = pd.read_csv('D:/jupyter_notebooks/GoProtect/dataset/raw_data.csv')
display(raw_data.sample(10, random_state=42))
raw_data.info()

,school_id,name
711,61,"СПб АНО ФСО ""КФКнК ""Звезда"""
440,143,"Республика Татарстан, РСШОР г.Казань"
525,111,ЧОУ ДОД ДЮСШ «Невский лед»
722,56,"Республика Карелия, МОУ ДО ДЮСШ 2"
39,275,"МО Светлановское, ООО «Ледовая история»"
290,196,ГБУ СШОР по ледовым видам спорта
300,192,ГБУ СШОР Колпинского района
333,179,"СШ по ФКиХ, Краснодарский край"
208,212,"Омская область, БУ ""СШОР № 35"""
136,244,"ГБУ ДО МАФК, школа Хрустальный"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name       895 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.1+ KB


Посчитаем, сколько `school_id` совпадает в двух датасетах.

In [4]:
# Количество уникальных id
print('Уникальных id в reference:', reference['school_id'].nunique())
print('Уникальных id в raw_data: ', raw_data['school_id'].nunique())

# Конвертируем в сеты
set1 = set(reference['school_id'])
set2 = set(raw_data['school_id'])

# Найдём количество пересечений
common_values = set1.intersection(set2)
print('Пересечений в id:         ', len(common_values))

Уникальных id в reference: 306
Уникальных id в raw_data:  264
Пересечений в id:          264


### Наблюдения

В датасете с сырыми данными содержатся все id школ, которые есть в датасете с референсными значениями. Это значит, что мы можем использовать датасет с референсными значениями для обучения модели, а датасет с сырыми данными для валидации. Для тестирования собран отдельный датасет, хранящийся у заказчика проекта.

## 2. Исследовательский анализ данных

Нам необходимо удостовериться, что список эталонных названий не содержит дубликатов.

Сперва очистим названия школ и регионов от лишних пробелов до и после. Затем поищем явные дубликаты и дубликаты по совокупности полей.

In [5]:
reference['name'] = reference['name'].str.strip()
reference['region'] = reference['region'].str.strip()

print('Явные дубликаты:          ', reference.duplicated().sum())
print('Дубликаты в name + region:', reference.duplicated(['name', 'region']).sum())

Явные дубликаты:           0
Дубликаты в name + region: 1


Удаление white spaces позволило найти дубликаты. Дубликаты в эталонном списке негативно повлияют на качество модели, в рамках исследования мы их удалим, а также обратим внимание заказчика на факт их наличия.

Теперь рассмотрим отдельно регионы.

In [6]:
# Создаём словарь для хранения потенциальных дубликатов
potential_duplicates = {}

# Создаём список с уникальными названиями регионов
regions = reference['region'].unique().tolist()

# Проходим в цикле по каждому региону
for region in regions:
    # Ищем потенциальные дубликаты с расстоянием Левенштейна больше 92
    matches = process.extract(region, regions, scorer=fuzz.ratio, score_cutoff=92)
    potential_duplicates[region] = matches

for region, matches in potential_duplicates.items():
    if len(matches) > 1:
        print(f"Potential duplicates for '{region}':")
        for match in matches:
            if region not in match:
                print(match)
        print()

Potential duplicates for 'Костромская бласть':
('Костромская область', 97.2972972972973, 22)

Potential duplicates for 'Костромская область':
('Костромская бласть', 97.2972972972973, 21)

Potential duplicates for 'Республика Карелия':
('Республика Корелия', 94.44444444444444, 33)

Potential duplicates for 'Удмуртская Республика':
('Удмуртская республика', 95.23809523809523, 72)

Potential duplicates for 'Республика Корелия':
('Республика Карелия', 94.44444444444444, 30)

Potential duplicates for 'Чувашская республика':
('Чувашская Республика', 95.0, 66)

Potential duplicates for 'Чувашская Республика':
('Чувашская республика', 95.0, 56)

Potential duplicates for 'Удмуртская республика':
('Удмуртская Республика', 95.23809523809523, 32)



В эталонном списке найдены неявные дубликаты: разные варианты написания одного и того же названия и опечатки. Устраним их и рассмотрим, повлияло ли это на наличие дубликатов в таблице.

In [7]:
reference['region'] = reference['region'].str.replace('Костромская бласть', 'Костромская область')
reference['region'] = reference['region'].str.replace('Корелия', 'Карелия')
reference['region'] = reference['region'].str.replace('республика', 'Республика')

# Дополнительно удалим No для обозначения номера
reference['name'] = reference['name'].str.replace('No', '')

print('Явные дубликаты:          ', reference.duplicated().sum())
print('Дубликаты в name + region:', reference.duplicated(['name', 'region']).sum())

Явные дубликаты:           0
Дубликаты в name + region: 3


Исправление опечаток позволило найти явные дубликаты.

In [8]:
schools_by_region = (
    reference
    .pivot_table(index='region', values='name', aggfunc='count')
    .sort_values(by='name', ascending=False)
)
schools_by_region[schools_by_region['name']>1]

,name
region,
Санкт-Петербург,64
Москва,29
Московская область,15
Свердловская область,10
Республика Татарстан,9
Нижегородская область,8
Ставропольский край,7
Челябинская область,7
Тверская область,7


Рассмотрим потенциальные дубликаты названий в каждом регионе.

In [9]:
# Создаём словарь для хранения потенциальных дубликатов
potential_duplicates = {}

# Создаём список с уникальными названиями регионов
regions = reference['region'].unique().tolist()

# Создаём список с уникальными названиями школ
schools = reference['name'].unique().tolist()

# Проходим в цикле по каждому региону и названию
for region in regions:
    region_schools = reference[reference['region']==region]['name'].unique().tolist()
    print(region)
    for school in region_schools:
        # Ищем потенциальные дубликаты с расстоянием Левенштейна больше 90
        matches = process.extract(school, region_schools, scorer=fuzz.ratio, score_cutoff=90)
        potential_duplicates[school] = matches
    
    for school, matches in potential_duplicates.items():
        if len(matches) > 1:
            print(f"Potential duplicates for '{school}':")
            for match in matches:
                if school not in match:
                    print(match)
            print()
    print()

Московская область

Ямало-Ненецкий АО

Республика Татарстан

Санкт-Петербург
Potential duplicates for 'СШОР по фигурному катанию на коньках':
('ЦОП по фигурному катанию на коньках', 92.95774647887323, 37)

Potential duplicates for 'ЦОП по фигурному катанию на коньках':
('СШОР по фигурному катанию на коньках', 92.95774647887323, 31)


Республика Крым
Potential duplicates for 'СШОР по фигурному катанию на коньках':
('ЦОП по фигурному катанию на коньках', 92.95774647887323, 37)

Potential duplicates for 'ЦОП по фигурному катанию на коньках':
('СШОР по фигурному катанию на коньках', 92.95774647887323, 31)


Рязанская область
Potential duplicates for 'СШОР по фигурному катанию на коньках':
('ЦОП по фигурному катанию на коньках', 92.95774647887323, 37)

Potential duplicates for 'ЦОП по фигурному катанию на коньках':
('СШОР по фигурному катанию на коньках', 92.95774647887323, 31)


Свердловская область
Potential duplicates for 'СШОР по фигурному катанию на коньках':
('ЦОП по фигурному катанию

In [10]:
# Выведем все явные дубликаты
duplicates = reference[reference.duplicated(['name', 'region'], keep=False)]
duplicates

,school_id,name,region
38,39,Голубева,Костромская область
39,40,Голубева,Костромская область
66,67,Керриган,Республика Карелия
67,68,Керриган,Республика Карелия
185,186,СШ № 6,Мурманская область
186,187,СШ № 6,Мурманская область


In [11]:
# Создадим список для замены дублирующихся school_id
replacement_dict = {}
for name_region, group in duplicates.groupby(['name', 'region']):
    primary_school_id = group.iloc[0]['school_id']
    duplicate_school_ids = group.iloc[1:]['school_id']
    for duplicate_id in duplicate_school_ids:
        replacement_dict[duplicate_id] = primary_school_id
replacement_dict

{40: 39, 68: 67, 187: 186}

Для всех ранее найденных дубликатов в эталонной таблице необходимо произвести замены id в таблице с сырыми данными.

In [12]:
# Произведём замену в пользовательских данных
raw_data['school_id'] = raw_data['school_id'].replace(replacement_dict)

# Удаление явных дубликатов
reference = (reference
             .drop_duplicates(subset=['name', 'region'], keep='first')
             .reset_index(drop=True)
            )

### Результаты исследовательского анализа

Обнаружены и удалены явные дубликаты в эталонной таблице:
* 187	СШ № 6	Мурманская область
* 40	Голубева	Костромская область
* 68	Керриган	Республика Карелия

В таблице с вариантами пользовательского ввода эти id заменены на корректные из эталонной таблицы.

Визуальный анализ показал наличие потенциальных дубликатов в названиях школ в разных регионах. Трудность анализа заключается в наличии большого количества аббревиатур, а также вероятность того, что две разные школы имеют одинаковое название в рамках одного региона.

Для решения проблемы дубликатов в эталонном списке следует ввести дополнительное поле, однозначно идентифицирующее спортивную организацию. Например, ИНН/ОГРН/ОГРНИП.

## 3. Подготовка данных

Создадим столбец с эталонным названием, которое будут включать в себя регион и название школы.

In [13]:
# Создадим столбец с полным названием
reference['full_name'] = reference['region'] + ', ' + reference['name']
reference.tail(10)

,school_id,name,region,full_name
293,300,"ГАУ РО ""ХК ""Рязань-ВДВ""",Рязанская область,"Рязанская область, ГАУ РО ""ХК ""Рязань-ВДВ"""
294,301,"МБУ ""ЦФКиС г. Лобня""",Московская область,"Московская область, МБУ ""ЦФКиС г. Лобня"""
295,302,СШ №2,Республика Башкортостан,"Республика Башкортостан, СШ №2"
296,303,"ООО ""СетПоинт""",Москва,"Москва, ООО ""СетПоинт"""
297,304,СШ по ЗВС,Пензенская область,"Пензенская область, СШ по ЗВС"
298,305,Прогресс,Алтайский край,"Алтайский край, Прогресс"
299,609,"""СШ ""Гвоздика""",Удмуртская Республика,"Удмуртская Республика, ""СШ ""Гвоздика"""
300,610,"СШОР ""Надежда Губернии",Саратовская область,"Саратовская область, СШОР ""Надежда Губернии"
301,611,КФК «Айсберг»,Пермский край,"Пермский край, КФК «Айсберг»"
302,1836,"ООО ""Триумф""",Москва,"Москва, ООО ""Триумф"""


Создадим список аббревиатур, чтобы сделать замены.

In [14]:
def extract_abbreviations(school_names):
    abbreviations = []
    for name in school_names:
        matches = re.findall(r'\b[А-ЯЁ]+\b', name)
        abbreviations.extend(matches)
        abbreviations = [abb for abb in abbreviations if len(abb)>1]
    return set(abbreviations)

In [15]:
extract_abbreviations(reference['full_name'])

{'АНО',
 'АО',
 'ВДВ',
 'ВОСТОЧНАЯ',
 'ВСШОР',
 'ГАУ',
 'ГБОУ',
 'ГБУ',
 'ГЛАЙД',
 'ГУ',
 'ДО',
 'ДЮСОШ',
 'ДЮСШ',
 'ЗВС',
 'ИО',
 'ИП',
 'ИСТОРИЯ',
 'КО',
 'КФК',
 'ЛВС',
 'ЛЕДОВАЯ',
 'ЛЦ',
 'МАУ',
 'МАФКК',
 'МАФСУ',
 'МБОУ',
 'МБУ',
 'МБФСУ',
 'МКУ',
 'МО',
 'МОСГОРСПОРТ',
 'МУ',
 'НЛФК',
 'НО',
 'НП',
 'ОКСШОР',
 'ОЛИМПИЯ',
 'ООО',
 'ООФФК',
 'ОРК',
 'РБ',
 'РК',
 'РО',
 'РОО',
 'РОФФКК',
 'РССШ',
 'РСШОР',
 'РСЯ',
 'РФ',
 'РФСОО',
 'РЦСП',
 'РЦСПЗВС',
 'СБС',
 'СИЯНИЕ',
 'СК',
 'СКА',
 'СТАРТАЙС',
 'СФК',
 'СФФК',
 'СШ',
 'СШОР',
 'ТИМ',
 'ТИТУЛ',
 'ТО',
 'ТОП',
 'УЗОР',
 'УОР',
 'УФФК',
 'ФАУ',
 'ФК',
 'ФКК',
 'ФОК',
 'ФФК',
 'ФФКК',
 'ФФККВО',
 'ФФККРК',
 'ФФККСК',
 'ХК',
 'ХМАО',
 'ЦЗВС',
 'ЦОП',
 'ЦСКА',
 'ЧУДО',
 'ШФК',
 'ЮНАРМЕЙЦЫ',
 'ЯНАО'}

In [16]:
# Создаем наборы аббревиатур
abbreviation_dict = {
    'АНО': ' автономная некоммерческая организация ',
    'ВСШОР': ' всеволжская спортивная школа олимпийского резерва ',
    'ГАУ': ' государственное автономное учреждение ',
    'ГБОУ': ' государственное бюджетное образовательное учреждение ',
    'ГБУ': ' государственное бюджетное учреждение ',
    'ГУ': ' государственное учреждение ',
    'ДО': ' дополнительного образования ',
    'ДЮСОШ': ' детско юношеская спортивная школа ',
    'ДЮСШ': ' детско юношеская спортивная школа ',
    'ЗВС': ' зимние виды спорта ',
    'ИП': ' индивидуальный предприниматель ',
    'КФК': ' коллектив физической культуры ',
    'ЛВС': ' ледовые виды спорта ',
    'ЛЦ': ' ледовый центр ',
    'МАУ': ' муниципальное автономное учреждение ',
    'МАФКК': ' московская академия фигурного катания на коньках ',
    'МАФСУ': ' муниципальное автономное физкультурно спортивное учреждение ',
    'МБОУ': ' муниципальное бюджетное общеобразовательное учреждение ',
    'МБФСУ': ' муниципальное бюджетное физкультурно спортивное учреждение ',
    'МБУ': ' муниципальное бюджетное учреждение ',
    'МКУ': ' муниципальное казенное учреждение ',
    'МУ': ' муниципальное учреждение ',
    'НЛФК': ' национальная лига фигурного катания ',
    'НО': ' некоммерческая организация ',
    'НП': ' некоммерческое портнерство ',
    'ОКСШОР': 'областная комплексная спортивная школа олимпийского резерва',
    'ООО': 'общество с ограниченной ответственностью',
    'РБ': ' республика башкортостан ',
    'РОО': ' региональная общественная организация ',
    'РОФФКК': ' ростовская областная федерация фигурного катания ',
    'РССШ': ' республиканская специализированная спортивная школа ',
    'РСШОР': ' республиканская спортивная школа олимпийского резерва ',
    'РФСОО': ' региональная физкультурно спортивная общественная организация ',
    'РЦСПЗВС': ' региональный центр спортивной подготовки по зимним видам спорта ',
    'РЦСП': ' региональный центр спортивной подготовки ',
    'ЦСКА': ' центральный спортивный клуб армии ',
    'СПб': ' санкт петербург ',
    'СПБ': ' санкт петербург ',
    'СКА': ' спортивный клуб армии ',
    'СК': ' спортивный клуб ',
    'СФК': ' спортивно физкультурный комплекс ',
    'СШОР': ' спортивная школа олимпийского резерва ',
    'СШ': ' спортивная школа ',
    'УОР': ' училище олимпийского резерва ',
    'ФАУ': ' федеральное автономное учреждение ',
    'ФОК': ' физкультурно оздоровительный комплекс ',
    'ФФК': ' федерация фигурного катания ',
    'ФКК': ' фигурного катания на коньках ',
    'ХК': ' хоккейный клуб ',
    'ХМАО': ' ханты мансийский автономный округ ',
    'ЦЗВС': ' центр зимних видов спорта ',
    'ЦОП': 'центр образовательных программ',
    'ШФК': ' школа фигурного катания ',
    'ЯНАО': ' ямало ненецкий автономный округ ',
    'АО': ' автономный округ '
}

In [17]:
# Функция для удаления аббревиатур
def remove_abbreviations(text, abbreviation_dict):
    for abbr in abbreviation_dict.keys():
        text = text.replace(abbr, '')
    return text.strip()

# Функция для замены аббревиатур
def replace_abbreviations(text, abbreviation_dict):
    for abbr, full_form in abbreviation_dict.items():
        text = text.replace(abbr, full_form)
    return text.strip()

# Функция для подготовки текста
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
        
    # Очистка от остальных спецсимволов
    text = re.sub(r'[^0-9a-zA-Zа-яёЁ]', ' ', text)
        
    # Токенизация
    tokens = [token for token in text.split() if (token.isnumeric() or len(token) > 1)]
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

Создадим признаки, по которым будем проверять сходство.

In [18]:
search_data = reference.copy(deep=True)

# Предобработка референсных данных
search_data['full_name_no_ab'] = (reference['full_name']
                                  .apply(lambda x: remove_abbreviations(x, abbreviation_dict))
                                  .apply(lambda x: preprocess_text(x))
                                 )

search_data['full_name_full_ab'] = (reference['full_name']
                                    .apply(lambda x: replace_abbreviations(x, abbreviation_dict))
                                    .apply(lambda x: preprocess_text(x))
                                   )

search_data['short_name'] = reference['name'].apply(lambda x: preprocess_text(x))

search_data['full_name'] = reference['full_name'].apply(lambda x: preprocess_text(x))

search_data = search_data.drop(['name','region'], axis=1)
search_data.tail(10)

,school_id,full_name,full_name_no_ab,full_name_full_ab,short_name
293,300,рязанская область гау ро хк рязань вдв,рязанская область ро рязань вдв,рязанская область государственное автономное у...,гау ро хк рязань вдв
294,301,московская область мбу цфкис лобня,московская область цфкис лобня,московская область муниципальное бюджетное учр...,мбу цфкис лобня
295,302,республика башкортостан сш 2,республика башкортостан 2,республика башкортостан спортивная школа 2,сш 2
296,303,москва ооо сетпоинт,москва сетпоинт,москва общество ограниченной ответственностью ...,ооо сетпоинт
297,304,пензенская область сш по звс,пензенская область по,пензенская область спортивная школа по зимние ...,сш по звс
298,305,алтайский край прогресс,алтайский край прогресс,алтайский край прогресс,прогресс
299,609,удмуртская республика сш гвоздика,удмуртская республика гвоздика,удмуртская республика спортивная школа гвоздика,сш гвоздика
300,610,саратовская область сшор надежда губернии,саратовская область надежда губернии,саратовская область спортивная школа олимпийск...,сшор надежда губернии
301,611,пермский край кфк айсберг,пермский край айсберг,пермский край коллектив физической культуры ай...,кфк айсберг
302,1836,москва ооо триумф,москва триумф,москва общество ограниченной ответственностью ...,ооо триумф


### Результаты подготовки данных

Мы создали дополнительные признаки:
* название + регион, аббревиатуры удалены;
* название + регион, аббревиатуры расшифрованы;
* название + регион как есть;
* только название.

А также подготовили текст для работы моделей: привели к нижнему регистру, очистили от спецсимволов и токенизировали.

## 4. Создание функций для применения моделей

Создадим функцию, которая будет создавать эмбеддинги признаков из эталонной таблицы. На вход функция будет принимать инициализированную языковую модель.

In [19]:
def create_reference_embeddings(model):
    global full_name_embed, full_name_no_ab_embed, full_name_full_ab_embed, short_name_embed
    full_name_embed = model.encode(search_data['full_name'].values, convert_to_tensor=True)
    full_name_no_ab_embed = model.encode(search_data['full_name_no_ab'].values, convert_to_tensor=True)
    full_name_full_ab_embed = model.encode(search_data['full_name_full_ab'].values, convert_to_tensor=True)
    short_name_embed = model.encode(search_data['short_name'].values, convert_to_tensor=True)

Далее создадим функцию, которая будет проводить семантический анализ по всем вариантам пользовательского ввода и эталонных названий. На вход функция принимает строку-запрос (пользовательский ввод) и максимальное количество кандидатов.

Пояснение к параметру `max_results`: функция отберет указанное количество значений с наибольшей степенью сходства, а затем классифицирует степень сходства как высокую, среднюю или низкую. Если функция найдёт 2 кандидата с высокой степенью сходства и 3 со средней степенью сходства, будут отображены только кандидаты с высокой степенью сходства.

Значение `max_results` по умолчанию = 5.

In [20]:
def school_name_semantic_search(query: str, max_results=5):
    query_no_ab = preprocess_text(remove_abbreviations(query, abbreviation_dict))
    query_full_ab = preprocess_text(replace_abbreviations(query, abbreviation_dict))
    query = preprocess_text(query)
    
    query_embed = model.encode(query, convert_to_tensor=True)
    query_no_ab_embed = model.encode(query_no_ab, convert_to_tensor=True)
    query_full_ab_embed = model.encode(query_full_ab, convert_to_tensor=True)
    
    queries = [query_embed, query_no_ab_embed, query_full_ab_embed]
    references = [full_name_embed, full_name_no_ab_embed, full_name_full_ab_embed, short_name_embed]
    
    search_results = []
    
    for query_embeddings in queries:
        for reference_embeddings in references:
            search_result = util.semantic_search(query_embeddings,
                                                 reference_embeddings,
                                                 top_k=3)[0]
            search_results.append(search_result)
    
    flattened_results = [item for sublist in search_results for item in sublist]
  
    unique_results = {}
    for item in flattened_results:
        item_id = item['corpus_id']
        if item_id not in unique_results or item['score'] > unique_results[item_id]['score']:
            unique_results[item_id] = item
    
    top_results = heapq.nlargest(max_results, unique_results.values(), key=lambda x: x['score'])
    
    filtered_results_high = [result for result in top_results if result['score'] >= 0.80]
    filtered_results_medium = [result for result in top_results if (result['score'] >= 0.6) and (result['score'] < 0.80)]
    
    if len(filtered_results_high) == 0 and len(filtered_results_medium) == 0:
        final_result = []
        
    elif len(filtered_results_high) != 0:
        filtered_results_ids = [result['corpus_id'] for result in filtered_results_high]
        filtered_results_rows = reference[reference.index.isin(filtered_results_ids)]
        final_result = filtered_results_rows['school_id'].tolist()
        
    else:
        filtered_results_ids = [result['corpus_id'] for result in filtered_results_medium]
        filtered_results_rows = reference[reference.index.isin(filtered_results_ids)]
        final_result = filtered_results_rows['school_id'].tolist()
    
    return final_result

Создадим функцию, которая будет проверять вхождение реального `school_id` в список id кандидатов.

In [21]:
def is_in_candidates(row, column):
    if row['school_id'] in row[column]:
        return 1
    else:
        return 0

Функция для демонстрации результата работы модели.

In [22]:
def demo(query, max_results):
    display(reference[reference['school_id'].isin(school_name_semantic_search(query , max_results=max_results))])

Все требуемые функции готовы. Можно применять модели.

## 5. Оценка качества моделей

Мы будем оценивать качество моделей метрикой accuracy. При этом мы замерим процент правильных ответов модели при максимальном числе кандидатов 1 и 5.

In [23]:
# Создадим таблицу для хранения метрик
model_metrics = pd.DataFrame({'model_name': ['sberbank-ai/sbert_large_nlu_ru', 'LaBSE'],
                              'accuracy_max_1': [0, 0],
                              'accuracy_max_5': [0, 0]
                             })


### Модель `sberbank-ai/sbert_large_nlu_ru`

In [24]:
# Инициализируем модель
model = SentenceTransformer('sberbank-ai/sbert_large_nlu_ru')

In [25]:
# Создадим эмбеддинги для референсных значений
create_reference_embeddings(model)

# Обтерем кандидатов для max_results 1 и 5
raw_data['candidates_sber_max_1'] = raw_data['name'].apply(lambda name: school_name_semantic_search(name, max_results=1))
raw_data['candidates_sber_max_5'] = raw_data['name'].apply(lambda name: school_name_semantic_search(name, max_results=5))

# Проверим, есть ли реальный school_id в списке кандидатов 
raw_data['target_sber_max_1'] = raw_data.apply(lambda row: is_in_candidates(row, 'candidates_sber_max_1'), axis=1)
raw_data['target_sber_max_5'] = raw_data.apply(lambda row: is_in_candidates(row, 'candidates_sber_max_5'), axis=1)

# Рассчитаем долю правильных ответов модели
accuracy_max_1 = raw_data['target_sber_max_1'].sum() / len(raw_data['target_sber_max_1'])
accuracy_max_5 = raw_data['target_sber_max_5'].sum() / len(raw_data['target_sber_max_5'])

# Добавим метрики в сравнительную таблицу
model_metrics.loc[0, 'accuracy_max_1'] = accuracy_max_1
model_metrics.loc[0, 'accuracy_max_5'] = accuracy_max_5

### Модель `LaBSE`

In [26]:
# Инициализируем модель
model = SentenceTransformer('LaBSE')

In [27]:
# Создадим эмбеддинги для референсных значений
create_reference_embeddings(model)

# Обтерем кандидатов для max_results 1 и 5
raw_data['candidates_labse_max_1'] = raw_data['name'].apply(lambda name: school_name_semantic_search(name, max_results=1))
raw_data['candidates_labse_max_5'] = raw_data['name'].apply(lambda name: school_name_semantic_search(name, max_results=5))

# Проверим, есть ли реальный school_id в списке кандидатов 
raw_data['target_labse_max_1'] = raw_data.apply(lambda row: is_in_candidates(row, 'candidates_labse_max_1'), axis=1)
raw_data['target_labse_max_5'] = raw_data.apply(lambda row: is_in_candidates(row, 'candidates_labse_max_5'), axis=1)

# Рассчитаем долю правильных ответов модели
accuracy_max_1 = raw_data['target_labse_max_1'].sum() / len(raw_data['target_labse_max_1'])
accuracy_max_5 = raw_data['target_labse_max_5'].sum() / len(raw_data['target_labse_max_5'])

# Добавим метрики в сравнительную таблицу
model_metrics.loc[1, 'accuracy_max_1'] = accuracy_max_1
model_metrics.loc[1, 'accuracy_max_5'] = accuracy_max_5

### Сравнение моделей

Посмотрим на результаты работы обеих моделей. В списках кандидатов представлены `school_id` для школ с наиболее схожим наименованием.

In [28]:
# Выведем полученные результаты на экран
raw_data.sample(3, random_state=42).T

,711,440,525
school_id,61,143,111
name,"СПб АНО ФСО ""КФКнК ""Звезда""","Республика Татарстан, РСШОР г.Казань",ЧОУ ДОД ДЮСШ «Невский лед»
candidates_sber_max_1,[284],[143],[297]
candidates_sber_max_5,"[52, 192, 277, 284, 285]","[143, 153, 205, 213, 230]","[74, 111, 174, 175, 297]"
target_sber_max_1,0,1,0
target_sber_max_5,0,1,1
candidates_labse_max_1,[277],[143],[111]
candidates_labse_max_5,[277],"[130, 143, 153, 201, 230]",[111]
target_labse_max_1,0,1,1
target_labse_max_5,0,1,1


Сравним метрики качества моделей при разных значениях `max_results`.

In [29]:
# Выведем метрики моделей
model_metrics

,model_name,accuracy_max_1,accuracy_max_5
0,sberbank-ai/sbert_large_nlu_ru,0.671508,0.858101
1,LaBSE,0.823464,0.916201


Ожидаемо, модели показали лучше результаты при более высоком значении `max_results`, когда правильным ответом считается тот, где хотя бы один из предложенных моделью вариантов корректен.

## 6. Результаты исследования
### Обработка и подготовка данных
В датасете с эталонными названиями школ найдено 3 дубликата. Они появились из-за опечаток и лишних пробелов в конце строк.

|Оригинальная запись|Дубликат|Причина|
|--|--|--|
|186 СШ № 6 Мурманская область|187 СШ № 6 Мурманская область| white space|
|39 Голубева Костромская область|40 Голубева Костромская бласть|опечатка|
|67 Керриган Республика Карелия|68 Керриган Республика Корелия|опечатка|

Прежде чем использовать модель, рекомендуется произвести соответствующие замены и удалить дубликаты в файлах заказчика.

**Предложения по улучшению качества данных:**
1. Добавить идентификатор, который будет однозначно определять школу. В качестве такого идентификатора может быть ОГРН/ОГРНИП. Данная информация находится в публичном доступе и используется для проверки контрагентов.
2. Использовать в качестве эталонных названий полные наименования организаций, так как наличие аббревиатур, не использующихся повсеместно или имеющих несколько значений, значительно затрудняет работу модели. Примером таких аббревиатур могут служить:
    * "РСЯ" - не является общеизвестной и общепринятой аббревиатурой;
    * "МО", "АО" - имеют более одного значения.
    
Мы создали дополнительные признаки:
* название + регион, аббревиатуры удалены;
* название + регион, аббревиатуры расшифрованы;
* название + регион как есть;
* только название.

А также подготовили текст для работы моделей: привели к нижнему регистру, очистили от спецсимволов и токенизировали.

### Применение моделей и оценка их качества
В качестве моделей были выбраны:
1. **sberbank-ai/sbert_large_nlu_ru** - большая лингвистическая модель на основе BERT, разработанная Сбербанком, ориентирована на русский язык.
2. **LaBSE** - language-agnostic-модель на основе BERT, разработанная Google.

Метрикой качества выбрана accuracy - процент правильных ответов модели. При этом метрика замерялась на разном  количестве кандидатов.

Модель sberbank-ai/sbert_large_nlu_ru справилась неплохо:
* при `max_results = 1` метрика accuracy = 0.67;
* при `max_results = 5` метрика accuracy = 0.86.

Однако, модель, которая не зависит от языка, справилась гораздо лучше. Результаты модели LaBSE:
* при `max_results = 1` метрика accuracy = 0.82;
* при `max_results = 5` метрика accuracy = 0.92.

То есть, если задать `max_results = 5`, модель с 92%-вероятностью укажет корректное наименование в списке предложенных.

Пояснение к параметру `max_results`: функция отберет указанное количество значений с наибольшей степенью сходства, а затем классифицирует степень сходства как высокую, среднюю или низкую. Если функция найдёт 2 кандидата с высокой степенью сходства и 3 со средней степенью сходства, **будут отображены только кандидаты с высокой степенью сходства**. По умолчанию `max_results = 5`.

### Примеры работы модели LaBSE

In [30]:
demo('школа Елены Бережной', max_results=5)

,school_id,name,region,full_name
23,24,Школа ФК Е. Бережной,Санкт-Петербург,"Санкт-Петербург, Школа ФК Е. Бережной"


In [31]:
demo('АвАнГаРд', max_results=3)

,school_id,name,region,full_name
0,1,Авангард,Московская область,"Московская область, Авангард"
1,2,Авангард,Ямало-Ненецкий АО,"Ямало-Ненецкий АО, Авангард"


In [32]:
demo('Школа магии и волшебства Хогвартс, Великобритания', max_results=5)

,school_id,name,region,full_name


К сожалению, не нашлось школ, похожих на Хогвартс. Но и не должно было. Модель работает штатно.